# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [25]:
df = pd.read_csv("weatherData.csv")
df.head()


,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,pandan,14.05,124.17,84.24,75,1013,11.90,10
1,albany,42.60,-73.97,51.94,44,1019,1.99,100
2,hobart,-42.88,147.33,55.83,76,1001,6.93,75
3,saint-joseph,-21.37,55.62,68.00,77,1020,8.05,75
4,castro,-24.79,-50.01,59.67,57,1018,3.69,100


In [28]:
# ### Humidity Heatmap
# * Configure gmaps.
# * Use the Lat and Lng as locations and Humidity as the weight.
# * Add Heatmap layer to map.

In [41]:
gmaps.configure(api_key=g_key)

df = df.dropna()
humidity = df['humidity'].astype(float)

locations = df[['latitude', 'longitude']].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights= humidity, dissipating = False, max_intensity = max(df['humidity']), point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
mask1 = (df.temperature >= 70) & (df.temperature < 80)
mask2 = df.wind_speed < 10
mask3 = df.cloudiness == 0
mask4 = mask1 & mask2 & mask3

ideal_cities = df.loc[mask4].reset_index(drop=True)
ideal_cities

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,aksu,41.12,80.26,75.60,12,1012,3.49,0
1,jiazi,22.88,116.07,77.05,55,1017,7.29,0
2,la libertad,-2.23,-80.90,74.32,81,1013,7.20,0
3,namibe,-15.20,12.15,74.59,66,1013,6.76,0
4,tura,25.52,90.22,77.49,60,1012,3.33,0
5,zhangye,38.93,100.45,74.71,9,1015,6.24,0
6,jinchang,38.50,102.17,74.43,8,1013,1.88,0
7,wuwei,37.93,102.64,73.99,14,843,1.99,0
8,marzuq,14.40,46.47,75.42,45,1010,4.63,0
9,poum,-20.23,164.02,79.02,74,1012,6.20,0


In [44]:
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)

coordinates = cities[["latitude", "longitude"]]

markers = gmaps.marker_layer(coordinates)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [14]:
city_info = []

for indx, row in cities.iterrows():
    info_box = f"""
                <dl>
                <dt>Name</dt><dd>{row.cities}</dd>
                <dt>Temp</dt><dd>{row.temperature}</dd>
                <dt>Cloudiness</dt><dd>{row.cloudiness}%</dd>
                </dl>
                """
    
    city_info.append(info_box)

In [16]:
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)

coordinates = cities[["latitude", "longitude"]]

marker_layer = gmaps.marker_layer(coordinates, info_box_content=city_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
cities.head(1)

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,aksu,41.12,80.26,75.6,12,1012,3.49,0


In [54]:
hotel_df = df

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    #get lat, lng from df
    lat = row['latitude']
    lng = row['longitude']
    
    params['location'] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['cities']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: pandan.
Closest hotel is Bonifacio Lodge.
------------
Retrieving Results for Index 1: albany.
Missing field/result... skipping.
------------
Retrieving Results for Index 2: hobart.
Closest hotel is St Ives Apartments.
------------
Retrieving Results for Index 3: saint-joseph.
Closest hotel is "Plantation Bed and Breakfast.
------------
Retrieving Results for Index 4: castro.
Closest hotel is CHACARA BAILLY.
------------
Retrieving Results for Index 5: hilo.
Closest hotel is Hilo Hawaiian Hotel.
------------
Retrieving Results for Index 6: crossville.
Closest hotel is Hampton Inn Crossville.
------------
Retrieving Results for Index 7: aksu.
Closest hotel is Pudong Holiday Hotel.
------------
Retrieving Results for Index 8: saint anthony.
Closest hotel is Holiday Inn Express Roseville-St. Paul.
------------
Retrieving Results for Index 9: hithadhoo.
Closest hotel is Scoop Guest House.
------------
Retrieving Results for Index 10: thompson.
Closest hotel 

Closest hotel is Ellie's Villa.
------------
Retrieving Results for Index 87: shingu.
Closest hotel is 新宮ユーアイホテル.
------------
Retrieving Results for Index 88: ca mau.
Closest hotel is Khách Sạn Mường Thanh Luxury Cà Mau.
------------
Retrieving Results for Index 89: praya.
Closest hotel is LT Aparthotel.
------------
Retrieving Results for Index 90: brasileia.
Closest hotel is Pousada Hotel Tobias.
------------
Retrieving Results for Index 91: shahrud.
Closest hotel is Royal Hotel.
------------
Retrieving Results for Index 92: saldanha.
Closest hotel is Blue Bay Lodge.
------------
Retrieving Results for Index 93: meghri.
Closest hotel is Haer B&B.
------------
Retrieving Results for Index 94: maragogi.
Closest hotel is Hotel Praia Dourada.
------------
Retrieving Results for Index 95: port elizabeth.
Closest hotel is 39 On Nile Guest House.
------------
Retrieving Results for Index 96: broome.
Closest hotel is Chenango Valley State Park.
------------
Retrieving Results for Index 97: 

Closest hotel is Comfort Suites Downtown.
------------
Retrieving Results for Index 174: santiago del estero.
Closest hotel is Hotel Urbano.
------------
Retrieving Results for Index 175: saskylakh.
Closest hotel is Saskylakh Aeroport.
------------
Retrieving Results for Index 176: hvolsvollur.
Closest hotel is Aurora Lodge Hotel.
------------
Retrieving Results for Index 177: roi et.
Closest hotel is THE NINE HOTEL ROI-ET.
------------
Retrieving Results for Index 178: jinchang.
Closest hotel is Mingshihui Jinchang International Hotel.
------------
Retrieving Results for Index 179: comodoro rivadavia.
Closest hotel is Austral Hotel.
------------
Retrieving Results for Index 180: busca.
Closest hotel is Trattoria Fermata Ceretto.
------------
Retrieving Results for Index 181: pevek.
Closest hotel is Severnoye Zoloto.
------------
Retrieving Results for Index 182: teahupoo.
Closest hotel is Vanira Lodge.
------------
Retrieving Results for Index 183: vozrozhdeniye.
Closest hotel is Kipa

Closest hotel is Ritz Lauca.
------------
Retrieving Results for Index 260: muros.
Closest hotel is Apartment Av. Castelao.
------------
Retrieving Results for Index 261: tasiilaq.
Closest hotel is The Red House.
------------
Retrieving Results for Index 262: marzuq.
Missing field/result... skipping.
------------
Retrieving Results for Index 263: sinnamary.
Closest hotel is ALETHEIA Appartement Sinnamary.
------------
Retrieving Results for Index 264: altamont.
Closest hotel is Microtel Inn & Suites by Wyndham Klamath Falls.
------------
Retrieving Results for Index 265: moville.
Closest hotel is EastSide Acres Mobile Home Park.
------------
Retrieving Results for Index 266: flin flon.
Closest hotel is Victoria Inn Flin Flon.
------------
Retrieving Results for Index 267: sitka.
Closest hotel is Westmark Sitka.
------------
Retrieving Results for Index 268: ghauspur.
Closest hotel is Press club.
------------
Retrieving Results for Index 269: namie.
Closest hotel is 福島いこいの村なみえ.
--------

Closest hotel is Hospitality Esperance.
------------
Retrieving Results for Index 347: elmira.
Closest hotel is Holiday Inn Elmira - Horseheads.
------------
Retrieving Results for Index 348: poum.
Closest hotel is Gîte kajeon.
------------
Retrieving Results for Index 349: dunedin.
Closest hotel is Scenic Hotel Southern Cross.
------------
Retrieving Results for Index 350: fojnica.
Closest hotel is Rooms Andric.
------------
Retrieving Results for Index 351: pacet.
Closest hotel is Hotel Istana.
------------
Retrieving Results for Index 352: gimli.
Closest hotel is Lakeview Gimli Resort.
------------
Retrieving Results for Index 353: naryan-mar.
Closest hotel is Hostel Agat.
------------
Retrieving Results for Index 354: barcelona.
Closest hotel is Axel Hotel Barcelona.
------------
Retrieving Results for Index 355: yarim.
Closest hotel is بقاله ابو محظار.
------------
Retrieving Results for Index 356: disna.
Missing field/result... skipping.
------------
Retrieving Results for Index 

Closest hotel is Pousada Porto De Canoas.
------------
Retrieving Results for Index 434: balikpapan.
Closest hotel is Swiss-Belhotel Balikpapan.
------------
Retrieving Results for Index 435: sakakah.
Closest hotel is Fakhamat Al Orjoana Apartment.
------------
Retrieving Results for Index 436: kulhudhuffushi.
Closest hotel is Haajy Guest House 1.
------------
Retrieving Results for Index 437: bang saphan.
Closest hotel is Ladawan Boutique Hotel.
------------
Retrieving Results for Index 438: motygino.
Closest hotel is Gostinitsa "Uyut".
------------
Retrieving Results for Index 439: yarmouth.
Closest hotel is The Inn At Cape Cod.
------------
Retrieving Results for Index 440: aklavik.
Closest hotel is Aklavik Inn.
------------
Retrieving Results for Index 441: deoria.
Closest hotel is Rajshree Inn.
------------
Retrieving Results for Index 442: buraydah.
Closest hotel is Mövenpick Hotel Qassim.
------------
Retrieving Results for Index 443: noumea.
Closest hotel is Hôtel Le Lagon.
---

Closest hotel is ООО "Гостиничный комплекс "Петропавловск".
------------
Retrieving Results for Index 521: egvekinot.
Missing field/result... skipping.
------------
Retrieving Results for Index 522: nusaybin.
Closest hotel is Aydinlar Hotel.
------------
Retrieving Results for Index 523: ridgetown.
Closest hotel is JOJO's Paradise Bed & Breakfast/SPA/Bakery Café.
------------
Retrieving Results for Index 524: tiznit.
Missing field/result... skipping.
------------
Retrieving Results for Index 525: dinar.
Closest hotel is Dinar Öğretmenevi.
------------
Retrieving Results for Index 526: aguimes.
Closest hotel is Hotel Villa de Aguimes.
------------
Retrieving Results for Index 527: skibbereen.
Closest hotel is The West Cork Hotel.
------------
Retrieving Results for Index 528: fairbanks.
Closest hotel is Westmark Fairbanks Hotel & Conference Center.
------------
Retrieving Results for Index 529: areosa.
Closest hotel is Hotel Flôr de Sal.
------------
Retrieving Results for Index 530: ya

In [58]:
hotel_df

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness,Hotel Name
0,pandan,14.05,124.17,84.24,75,1013,11.90,10,Bonifacio Lodge
1,albany,42.60,-73.97,51.94,44,1019,1.99,100,NaN
2,hobart,-42.88,147.33,55.83,76,1001,6.93,75,St Ives Apartments
3,saint-joseph,-21.37,55.62,68.00,77,1020,8.05,75,"""Plantation Bed and Breakfast"
4,castro,-24.79,-50.01,59.67,57,1018,3.69,100,CHACARA BAILLY
...,...,...,...,...,...,...,...,...,...
563,yenagoa,4.92,6.26,77.49,95,1012,3.33,41,Amba Hotels Limited
564,navahrudak,53.60,25.83,37.38,79,1008,7.65,100,Hotel Grazhina
565,manono,-7.30,27.42,69.24,77,1013,2.64,7,Hotel hewa bora
566,chumikan,54.72,135.31,33.89,95,998,6.89,100,Fortuna


In [62]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['lat', 'lng'], dtype='object')] are in the [columns]"

In [63]:
# Add marker layer ontop of heat map
locations = hotel_df[["latitude", "longitude"]]
markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…